In [ ]:

!pip install ipeadatapy  #Instalando o pacote 'ipeadatapy'

In [ ]:
# Carregando pacotes
import pandas as pd
import ipeadatapy
import matplotlib.pyplot as plt

# Obtendo serie com dados sobre emplacamento de veículos
emplacamentos = ipeadatapy.timeseries('FENABRAVE12_VENDAUTO12')
emplacamentos.reset_index(inplace=True)
emplacamentos = emplacamentos.loc[(emplacamentos["DATE"] >= "2018-01-01") & (emplacamentos["DATE"] < "2023-01-01"),
                                  "VALUE (Unidade)"]
emplacamentos.rename("emplacamentos", inplace=True)

# Obtendo serie com dados sobre IPI arrecadado sobre automóveis
ipi = ipeadatapy.timeseries('SRF12_IPIVEI12')
ipi.reset_index(inplace=True)
ipi = ipi.loc[(ipi["DATE"] >= "2018-01-01") & (ipi["DATE"] < "2023-01-01"),
                                  "VALUE (R$)"]
ipi.rename("ipi", inplace=True)

carros = pd.concat([emplacamentos.reset_index(drop=True),
                    ipi.reset_index(drop=True)],
                   axis=1)
carros

Relação de emplacamento x IPI mostrado em gráfico de dispersão.

In [ ]:
carros.plot.scatter(x='emplacamentos',y='ipi')
plt.title('Relação Emplacamento x IPI')
plt.show()


Conjunto de treinamento

In [ ]:
from sklearn.model_selection import train_test_split

treino, teste = train_test_split(carros, train_size=0.70, random_state=123)

Modelo linear da base treino

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

X_treino = sm.add_constant(treino['emplacamentos'])
y_treino = treino['ipi']

reg = LinearRegression()

modelo_estimado = reg.fit(X = X_treino, y = y_treino)

Dados observados e reta estimada pelo modelo linear

In [ ]:
X_teste = sm.add_constant(teste['emplacamentos'])
y_teste = teste['ipi']

previsao = modelo_estimado.predict(X = X_teste)
alfa = modelo_estimado.intercept_
beta = modelo_estimado.coef_[1]

teste.plot.scatter(x = 'emplacamentos', y = 'ipi', label = 'Observado')

x_prev = np.linspace(teste['emplacamentos'].min(), teste['ipi'].max(), 100).reshape(-1,1)

y_prev = alfa + beta * x_prev

plt.plot(x_prev, y_prev, color = 'red', label = 'Modelo Estimado')
plt.xlabel('Emplacamento')
plt.ylabel('IPI')
plt.title('Emplacamento x IPI')
plt.show()